# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 7.3.0
-- The CXX compiler identification is GNU 7.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home2/clinux01/.conda/envs/tp2/bin/python (found version "3.6.5") 
-- Found PythonLibs: /home2/clinux01/.conda/envs/tp2/lib/libpython3.6m.so
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home2/clinux01/Documents/SentimentAnalysisMetNum
--

In [2]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment


/home2/clinux01/Documents/SentimentAnalysisMetNum/notebooks
Python 3.6.5 :: Anaconda, Inc.


In [29]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
./._imdb_small.csv
imdb_small.csv
Cantidad de documentos: 12500


In [4]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,First of all this was not a three hour movie -...,neg,11606_4.txt
freq,6275,2,6322,2


In [30]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]

text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))



Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [6]:
from __future__ import division
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))


Class balance : 0.49493975903614457 pos 0.5050602409638554 neg


In [33]:
from sklearn.feature_extraction.text import CountVectorizer

print(text_train.shape)

vectorizer = CountVectorizer(max_df=0.99, min_df=0.001, max_features=15000)



vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

print(X_train.shape)

(6225,)
(6225, 10297)


In [20]:
import sentiment

# Usar PCA acá y pasarle el X_train modificado
pca = sentiment.PCA(500)
pca.fit(X_train, y_train)
pca_x_train = pca.transform(X_train)
pca_y_train = pca.transform(y_train)

pca_x_test = pca.transform(X_test)
pca_y_test = pca.transform(y_test)

clf = sentiment.KNNClassifier(1000)

clf.fit(X_train, y_train)


In [21]:
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test[:500])

acc = accuracy_score(y_test[:500], y_pred[:500])
print("Accuracy: {}".format(acc))

Accuracy: 0.662
CPU times: user 46.7 s, sys: 24.2 s, total: 1min 10s
Wall time: 1min 10s
